# Modelltrening for å gjette NVDBJira tasks

In [61]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import os
import sys
import re
import operator

## Hent inn jira-data
- fjern null-verdier og inspiser datasett

In [62]:
jira = os.path.join(os.getcwd(), 'data', 'jira.csv')
df = pd.read_csv(jira, usecols=["Description", "Component/s"])
df.columns = ['text', 'label']

In [63]:
df = df.dropna().reset_index(drop=True)
df

,text,label
0,Som bruker av Datafangst\r\n\r\nønsker jeg å k...,Datafangst
1,Jeg lager en sak her for å ha en utviklersak s...,Les-API
2,Ved import av objekter med sensitive egenskape...,Datafangst
3,Som bruker av Vegkart-eskport\r\n\r\nopplever ...,Eksport
4,Årsaken til at vi ser etter en måte å få et ko...,Les-API
...,...,...
919,TS-lint er deprecated: [https://medium.com/pal...,Vegkart
920,h2. Observasjon\r\n\r\nÅpne Vegkart. Legg til ...,Vegkart
921,h2. Observasjon\r\n\r\nPrøver å fjerne en egen...,Datafangst
922,h2. Observasjon\r\n\r\nHent opp url med id for...,Vegkart


In [5]:
df.iloc[920].text

"h2. Observasjon\r\n\r\nÅpne Vegkart. Legg til område Trondheim. Fjern Trondheim. Feil\r\n{noformat}\r\nUncaught TypeError: Cannot read property 'roadNetState' of undefined\r\n{noformat}"

## Mye rart i dataen her. Vask den!

In [66]:
spacy_model = 'nb_core_news_lg'
nlp = spacy.load(spacy_model)
nlp.add_pipe('sentencizer')

In [67]:
strip_bracket = r'(?<=\[).*?(?=\])'
strip_paren = r'(?<=\().*?(?=\))'
strip_tag = r'(?<=\<).*?(?=\>)'
valid_words = r'[^\s\-][\w \-]+[^\s\-]'

def flat_map(xs):
    ys = []
    for x in xs:
        ys.extend(x)
    return ys

def hyphen_no_date(word):
    if '-' in word:
        return word.split('-')[0].isalpha()
    return word.isalnum()

def clear_jira(text):
    text = re.sub(strip_bracket, '', text)
    text = re.sub(strip_paren, '', text)
    text = re.sub(strip_tag, '', text)
    text = ' '.join([w for w in text.splitlines() if len(w) > 0])
    text = ' '.join(re.findall(valid_words, text))    
    text = [w for w in text.split(' ') if hyphen_no_date(w)]
    text = [w for w in text if w[0] != '!']  # avoid image tags (!image-hash)
    
    text = ' '.join(text)
    return text

def clean(row):
    text = clear_jira(row.text)
    doc = nlp(text)
    row['text'] = text.lower()    
    row['label'] = row['label'].lower()

    return row

df = df.apply(clean, axis=1)

In [72]:
def get_texts_from_name(name, field='text'):
    return list(df[(df.label.str.contains(name.lower()))][field])

get_texts_from_name('vegkart')[:5]

['vi må vurdere nye funksjoner i vegkart og trenger estimat for utvikling av historisk data legge til felt for å skrive inn dato som vegkart skal vise data i forhold sensitive innlogging for å vise sensitive ved første forsøk på å vise sensitiv kopi av vegkart med faglige filter mulighet for å filtrere basert på andre for eksempel vise alle trafikkulykker på veg med fartsgrense 60 eller',
 'inn fra for fartsgrense filteret ønsker vi at man kan velge mindre og større nå er det kun og det er behov for å se vei med fartsgrense til og med 50km og tilsvarende fra og med 60km og legge til filtrering på større og mindre kontakt for å sjekke om ny filtrering dekker',
 'datakatalogen lite er basert på v2 og delar av funksjonaliteten er alt ute av den må oppdaterast og endrast til ser to alternativ oppdatere dagens produkt til v3 og reparere knekt funksjonalitet utvikle ny versjon basert på dakat i',
 'flytte changelog til ettersom gradle jobb i backend ikke er testes ikke funksjonelt',
 'ny fun

## Dette ser bedre ut! Rydd opp enda mer :)
- start med å fjerne labels med for få eksempler

In [75]:
data.label.value_counts()

datafangst          362
les-api             172
skriv-api           141
vegkart              88
rapportgenerator     72
nvdb-ind             69
eksport              16
database              3
datakontroll          1
Name: label, dtype: int64

In [77]:
THRESHOLD = 60
labels = [x for x, y in data.label.value_counts().items() if y > THRESHOLD]
data = data[data.label.isin(labels)]
data.label.value_counts()

datafangst          362
les-api             172
skriv-api           141
vegkart              88
rapportgenerator     72
nvdb-ind             69
Name: label, dtype: int64

## Kategoriser dataen for å være snill med maskinlæringsmodeller

In [ ]:
def categorize_labels(row):
    row['cats'] = { l : l == row.label for l in labels }
    return row

data = data.apply(categorize_labels, axis=1)
data.drop(['label'], axis=1, inplace=True)
data

## Splitt opp i trening og valideringssett

In [81]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)
test

,text,cats
41,av nvdb-6455 legg til sjekk at en kan søke på ...,"{'datafangst': True, 'les-api': False, 'skriv-..."
570,universell kunne bruke mellomrom til å velge f...,"{'datafangst': False, 'les-api': False, 'skriv..."
473,mail fra vi har registrert at det juni ble inn...,"{'datafangst': False, 'les-api': True, 'skriv-..."
731,det ser ut som det kun filtreres på så true vi...,"{'datafangst': False, 'les-api': True, 'skriv-..."
706,paus alle operasjoner mot solr dersom en eller...,"{'datafangst': False, 'les-api': False, 'skriv..."
...,...,...
589,bakgrunn datafangst og muligens andre klienter...,"{'datafangst': False, 'les-api': False, 'skriv..."
776,bakgrunn fra 2006 til i dag har vi brukt innle...,"{'datafangst': False, 'les-api': False, 'skriv..."
462,det er nå tydelig at det trengs et statusfelt ...,"{'datafangst': False, 'les-api': True, 'skriv-..."
260,observasjon har vegsystemreferanse med et vegs...,"{'datafangst': False, 'les-api': True, 'skriv-..."


In [29]:
def make_docs(data):   
    docs = []
    # nlp.pipe([texts]) is way faster than running nlp(text) for each text
    # as_tuples allows us to pass in a tuple, the first one is treated as text
    # the second one will get returned as it is.
    for doc, label in nlp.pipe(data, as_tuples=True):
        doc.cats = label
        # put them into a nice list
        docs.append(doc)
    
    return docs

In [30]:
from spacy.tokens import DocBin

In [33]:
_test = list(zip(test.text.tolist(), test.cats.tolist()))
_train = list(zip(train.text.tolist(), train.cats.tolist()))

train_docs = make_docs(_train)
test_docs = make_docs(_test)

In [34]:
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./data/train.spacy")

In [37]:
doc_bin = DocBin(docs=test_docs)
doc_bin.to_disk("./data/valid.spacy")

In [38]:
nlp = spacy.load("output/model-best")
text = ""
print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
while text != "quit":
    text = input("Skriv noe for : ")
    doc = nlp(text)
    print(doc.cats)

type : ‘quit’ to exit


Please enter example input:  vegkart


{'datafangst': 0.3518565595149994, 'les-api': 0.44431495666503906, 'skriv-api': 0.3457692861557007, 'vegkart': 0.5625988841056824, 'rapportgenerator': 0.4080062210559845, 'nvdb-ind': 0.38103434443473816}


Please enter example input:  quit


{'datafangst': 0.45040658116340637, 'les-api': 0.4717670679092407, 'skriv-api': 0.42499011754989624, 'vegkart': 0.4449041485786438, 'rapportgenerator': 0.4561050832271576, 'nvdb-ind': 0.45095202326774597}


In [39]:
x = {'datafangst': 0.3518565595149994, 'les-api': 0.44431495666503906, 'skriv-api': 0.3457692861557007, 'vegkart': 0.5625988841056824, 'rapportgenerator': 0.4080062210559845, 'nvdb-ind': 0.38103434443473816}

In [40]:
x

{'datafangst': 0.3518565595149994,
 'les-api': 0.44431495666503906,
 'skriv-api': 0.3457692861557007,
 'vegkart': 0.5625988841056824,
 'rapportgenerator': 0.4080062210559845,
 'nvdb-ind': 0.38103434443473816}